# Quick Price

In [ ]:
import pathlib

from ib_insync import IB, Stock, Contract, util
import datetime
from collections import defaultdict
import pandas as pd
import numpy as np
import asyncio

In [ ]:
from engine import Vars

In [ ]:
util.startLoop()

In [ ]:
# * INPUTS
MARKET = 'NSE'
SYMBOL = 'IOC'
DTE = 10

In [ ]:
# * SETTINGS
ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PAPER, ibp.CID

In [ ]:
df_opts = pd.read_pickle(pathlib.Path.cwd().joinpath('data', MARKET.lower(), 'df_opts.pkl'))
df_unds = pd.read_pickle(pathlib.Path.cwd().joinpath('data', MARKET.lower(), 'df_unds.pkl'))
df_chains = pd.read_pickle(pathlib.Path.cwd().joinpath('data', MARKET.lower(), 'df_chains.pkl'))

In [ ]:

async def quick_price_async(ib: IB, contract: Contract) -> pd.DataFrame:
    
    # Check for executeAsync engine
    if isinstance(contract, tuple):
        contract = contract[0]
    
    result = defaultdict(dict)
    
    ticks = await asyncio.gather(ib.reqHistoricalTicksAsync(
                                    contract=contract,
                                    startDateTime='',
                                    endDateTime=datetime.datetime.now(),
                                    numberOfTicks=1,
                                    whatToShow='Bid_Ask',
                                    useRth=False,
                                    ignoreSize=False),
                                ib.reqHistoricalTicksAsync(
                                    contract=contract,
                                    startDateTime='',
                                    endDateTime=datetime.datetime.now(),
                                    numberOfTicks=1,
                                    whatToShow='Trades',
                                    useRth=False,
                                    ignoreSize=False))
    
    # extract bid and ask price
    try:
        bid_ask = ticks[0][-1] # bid ask is not availble for Index securities!
        result['bid'] = bid_ask.priceBid
        result['ask'] = bid_ask.priceAsk
        
    except IndexError:
        print(f'\nNo bid-ask for {contract.localSymbol} of secType: {contract.secType}')
        result['bid'] = np.nan
        result['ask'] = np.nan       

    # extract last reported price
    try:
        # pick reported price if available
        result['last'] = [t.price for t in ticks[1] 
                      if not t.tickAttribLast.unreported][-1]
    except IndexError:
        # pick up last tick price
        
        try:
            result['last'] = ticks[1][-1].price
        except IndexError:
            result['last'] = np.nan
            
    
    # . build the df
    df_pr = pd.DataFrame([pd.Series(contract.conId, name='conId'),
                          pd.Series(contract.symbol, name='symbol'),
                          pd.Series(contract.localSymbol, name='localSymbol'),
                          pd.Series(result['bid'], name='bid', dtype='float64'), 
                          pd.Series(result['ask'], name='ask', dtype='float64'), 
                          pd.Series(result['last'], name='last', dtype='float64')]).T

    # . use bid-ask avg if last price is not available
    df_pr = df_pr.assign(price=df_pr['last'].combine_first(df_pr[['bid', 'ask']].mean(axis=1)))

    return df_pr

In [ ]:
IB().disconnect()

In [ ]:
%%time
# * MAKE TARGET OPTIONS

# . get chains
df_ch = df_chains[df_chains.symbol == SYMBOL]

# . get its undPrice
und_ct = df_unds[df_unds.symbol == SYMBOL].contract.iloc[0]

with IB().connect(HOST, PORT, CID) as ib:
    df_pr = ib.run(quick_price_async(ib, und_ct))

In [ ]:
df_pr

In [ ]:
%%time
# making quick_price_async run with create_tasks
import asyncio
async def main(ib, cts):
    tasks = [asyncio.create_task(quick_price_async(ib, c), name=c.symbol) for c in cts]
    return await asyncio.gather(*tasks)

cts = [c for c in df_unds.contract if c.symbol not in ['NIFTY50', 'BANKNIFTY']][:5] # !!! DATA LIMITER
with IB().connect(HOST, PORT, CID) as ib:
    ib.client.setConnectOptions('PACEAPI')
    dfs = ib.run(main(ib, cts))

In [ ]:
df = pd.concat(dfs, ignore_index=True)
df